In [ ]:
!pip install torchmetrics[multimodal]


In [ ]:
! pip install pyiqa

In [1]:
import torchmetrics
import torchmetrics.multimodal
from torchmetrics.multimodal import CLIPImageQualityAssessment as CLIPIQA
import pyiqa
import  glob
import os
import cv2
import torch
import lpips
import numpy as np
from PIL import Image
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
from skimage.restoration import estimate_sigma
from torchvision import transforms
#from clipiqa import CLIPIQA
#from musiq import MUSIQ
import torchmetrics

/home/kishore120/kishore_diffusion_env/lib/python3.8/site-packages/tqdm-4.66.2-py3.8.egg/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
musiq_metric = pyiqa.create_metric('musiq')

Loading pretrained model MUSIQ from /home/kishore120/.cache/torch/hub/pyiqa/musiq_koniq_ckpt-e95806b9.pth


In [3]:
clipiqa_metric = pyiqa.create_metric('clipiqa')

## Clip IQA score evaluation

In [4]:
# Set image folder path
import os
from PIL import Image
output_folder = '/srv/Data16tb/Kishore/ResShift/results/user_own_data_prosx_t2_inference_SR4'
# '/srv/Data16tb/Kishore/ResShift/results/Image_net_test_v1_64_64_32'
# "/srv/Data16tb/Kishore/ResShift/results/table6_journal_blid_face_restoration_faceIR"
save_path = os.path.join('/srv/Data16tb/Kishore/ResShift/results/',output_folder.split('/')[-1]+'_metrics.txt')
print('Metrics saving path ',save_path)

Metrics saving path  /srv/Data16tb/Kishore/ResShift/results/user_own_data_prosx_t2_inference_SR4_metrics.txt


In [5]:
clipiqa_scores = []

# Get the list of images in the folder (filter only valid image files)
output_images = [f for f in sorted(os.listdir(output_folder)) if f.endswith(('.jpg', '.png', '.jpeg'))]
print("total images to process",len(output_images))

# # Define image transformation to convert to tensor
transform = transforms.Compose([
    # transforms.Resize((256, 256)),
    transforms.ToTensor()  # Convert to PyTorch tensor
])

# Loop through images
for n,img_name in enumerate(output_images):
    # print(n,'---->>',img_name)
    img_path = os.path.join(output_folder, img_name)
    
    #print("Processing:", img_path)
    # Open image and convert to tensor
    img = Image.open(img_path).convert("RGB")
    if n==1:
        print("Pred img shape",img.size)
    img_tensor = transform(img).unsqueeze(0)  # Add batch dimension

    # Compute CLIPIQA Score
    with torch.no_grad():  # Disable gradients for inference
        # musiq_score = musiq_metric(img_tensor)
        clipiqa_score = clipiqa_metric(img_tensor)

    # musiq_scores.append(musiq_score.item())  # Convert tensor to float
    clipiqa_scores.append(clipiqa_score.item())

# Compute Average Score
# avg_musiq = np.mean(musiq_scores)
avg_clipiqa = np.mean(clipiqa_scores)

# print(f"\nAverage MUSIQ Score: {avg_musiq:.4f}")
print(f"\nAverage CLIPIQA Score: {avg_clipiqa:.4f}")
# Write to file
with open(save_path, 'a') as f:
    f.write(f"Average CLIPIQA Score: {avg_clipiqa:.4f}\n")

total images to process 1380
Pred img shape (1024, 1024)

Average CLIPIQA Score: 0.4967


## MusicIQA score evaluation

In [6]:
musiq_scores = []

# Get the list of images in the folder (filter only valid image files)
output_images = [f for f in sorted(os.listdir(output_folder)) if f.endswith(('.jpg', '.png', '.jpeg'))]
print("total images to process",len(output_images))

# # Define image transformation to convert to tensor
transform = transforms.Compose([
    # transforms.Resize((256, 256)),
    transforms.ToTensor()  # Convert to PyTorch tensor
])

# Loop through images
for n,img_name in enumerate(output_images):
    # print(n,'---->>',img_name)
    img_path = os.path.join(output_folder, img_name)
    #print("Processing:", img_path)
    # Open image and convert to tensor
    img = Image.open(img_path).convert("RGB")
    if n==1:
        print("Pred img shape",img.size)
    img_tensor = transform(img).unsqueeze(0)  # Add batch dimension

    # Compute CLIPIQA Score
    with torch.no_grad():  # Disable gradients for inference
        musiq_score = musiq_metric(img_tensor)
        # clipiqa_score = clipiqa_metric(img_tensor)
    musiq_scores.append(musiq_score.item())  # Convert tensor to float
    # clipiqa_scores.append(clipiqa_score.item())
# Compute Average Score
avg_musiq = np.mean(musiq_scores)
# avg_clipiqa = np.mean(clipiqa_scores)
print(f"\nAverage MUSIQ Score: {avg_musiq:.4f}")
# print(f"\nAverage CLIPIQA Score: {avg_clipiqa:.4f}")
# Write to file
with open(save_path, 'a') as f:
    f.write(f"Average MUSIQ Score: {avg_musiq:.4f}\n")
    print(f'saved to {save_path}')

total images to process 1380
Pred img shape (1024, 1024)

Average MUSIQ Score: 55.8166
saved to /srv/Data16tb/Kishore/ResShift/results/user_own_data_prosx_t2_inference_SR4_metrics.txt


## PSNR ,SSIM, LPIPS EVAUATION

In [7]:
def norm(array):
    return ((array-np.min(array)))/(np.max(array)-np.min(array))

In [8]:
base_inp_dir = '/srv/Data16tb/Kishore/ResShift/testdata/user_own_data/Pros_datas'
# '/srv/Data16tb/Kishore/ResShift/testdata/Synthetic_datas_from_repo/'
base_op_dir = '/srv/Data16tb/Kishore/ResShift/results/'

# gt_img_path = glob.glob(base_inp_dir+'/imagenet256_srx4/'+'**/*gt/')[0]
gt_img_path = glob.glob(base_inp_dir+'/T2_2d_prox_imgs/')[0]

# celeba512-faceir - for blind face restoration
# imagenet256_inpainting -  for inpainting task contains 2k images
# celeba256_face_inpainting - for face inpainting  - contains 2k face images

# pred_img_path = glob.glob(base_op_dir+'/Image_net_test_v1_64_64_32/')[0]
pred_img_path = glob.glob(base_op_dir+'user_own_data_prosx_t2_inference_SR4/')[0]

# table6_journal_blid_face_restoration_faceIR - prediction of celeba512-faceir blind face restoration
# table5_journal_inpaint_face - prediction of imagenet256 inpainting face 
# table5_inpaint_face_celeba_inference - prediction of face inpainting on celeba
# table4_inpaint_on_natural image_imagenet_inference-   inference of imagenet dataset natural inpainting

print("gt",gt_img_path,'\n',"Pred",pred_img_path)

# Define allowed extensions
allowed_extensions = {".jpg", ".jpeg", ".png"}

# Filter only valid image files
gt_images = sorted([f for f in sorted(os.listdir(gt_img_path)) if os.path.splitext(f)[1].lower() in allowed_extensions])
output_images = sorted([f for f in sorted(os.listdir(pred_img_path)) if os.path.splitext(f)[1].lower() in allowed_extensions])
assert len(gt_images)==len(output_images)
print(len(gt_images),len(output_images))
print(gt_images[0],output_images[0])

gt /srv/Data16tb/Kishore/ResShift/testdata/user_own_data/Pros_datas/T2_2d_prox_imgs/ 
 Pred /srv/Data16tb/Kishore/ResShift/results/user_own_data_prosx_t2_inference_SR4/
1380 1380
0000_T2_slice10.png 0000_T2_slice10.png


In [9]:
psnr_values = []
ssim_values = []
lpips_values = []

# Initialize LPIPS model
lpips_model = lpips.LPIPS(net='vgg').to("cuda" if torch.cuda.is_available() else "cpu")

# Define image transform for LPIPS
transform = transforms.Compose([
    transforms.ToTensor()
])

for gt_name, out_name in zip(gt_images, output_images):
    gt_path = os.path.join(gt_img_path, gt_name)
    out_path = os.path.join(pred_img_path, out_name)

    if not os.path.exists(out_path):
        print(f"Skipping {out_name} as it does not exist in output folder.")
        continue

    # Read images
    gt_img = cv2.imread(gt_path)
    out_img = cv2.imread(out_path)
    # gt_img = norm(cv2.imread(gt_path))
    # out_img = norm(cv2.imread(out_path))
    
    # print(gt_path,'\n',out_path)
    
    # print("GT img shape",gt_img.shape, "Pred img shape",out_img.shape)

    # Resize output image to match ground truth size
    out_img_resized = cv2.resize(out_img, (gt_img.shape[1], gt_img.shape[0]), interpolation=cv2.INTER_CUBIC)
    # print("Pred img resized", out_img_resized.shape)
    # print("GT image shape",gt_img.shape)
    
    

    # Convert to grayscale for SSIM calculation
    gt_gray = norm(cv2.cvtColor(gt_img, cv2.COLOR_BGR2GRAY))
    out_gray = norm(cv2.cvtColor(out_img_resized, cv2.COLOR_BGR2GRAY))

    # Compute PSNR and SSIM
    psnr_value = psnr(gt_img, out_img_resized, data_range=255)
    ssim_value = ssim(gt_gray, out_gray, data_range=255)

    psnr_values.append(psnr_value)
    ssim_values.append(ssim_value)

    # Convert images to tensors for LPIPS
    gt_tensor = transform(Image.fromarray(cv2.cvtColor(gt_img, cv2.COLOR_BGR2RGB))).unsqueeze(0).to("cuda" if torch.cuda.is_available() else "cpu")
    out_tensor = transform(Image.fromarray(cv2.cvtColor(out_img_resized, cv2.COLOR_BGR2RGB))).unsqueeze(0).to("cuda" if torch.cuda.is_available() else "cpu")

    # Compute LPIPS
    lpips_value = lpips_model(gt_tensor, out_tensor).item()
    lpips_values.append(lpips_value)

    #print(f"{out_name}: PSNR = {psnr_value:.2f}, SSIM = {ssim_value:.4f}, LPIPS = {lpips_value:.4f}")

# Compute average PSNR, SSIM, and LPIPS
avg_psnr = np.mean(psnr_values)
avg_ssim = np.mean(ssim_values)
avg_lpips = np.mean(lpips_values)

print(f"\nAverage PSNR: {avg_psnr:.2f}")
print(f"Average SSIM: {avg_ssim:.4f}")
print(f"Average LPIPS: {avg_lpips:.4f} (Lower is better)")

# normalize both the folders


Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [off]


/home/kishore120/kishore_diffusion_env/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/kishore120/kishore_diffusion_env/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: /home/kishore120/kishore_diffusion_env/lib/python3.8/site-packages/lpips/weights/v0.1/vgg.pth


/home/kishore120/kishore_diffusion_env/lib/python3.8/site-packages/lpips/lpips.py:107: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.load_state_dict(torch.load(model_pa


Average PSNR: 29.11
Average SSIM: 0.9999
Average LPIPS: 0.2117 (Lower is better)


In [10]:
with open(save_path, 'a') as f:
    f.write(f"Average PSNR Score: {avg_psnr:.4f}\n")
    f.write(f"Average SSIM Score: {avg_ssim:.4f}\n")
    f.write(f"Average LPIPS Score: {avg_lpips:.4f}\n")